## Edges filters

The purpose is to detect edges (=contours) on images. We will



### principle of 1D signal

Let's consider a simple signal 1D

In [ ]:
def buildSignal()->np.ndarray:
    a=50
    dec=30
    x=np.linspace(-a,a,1000)
    y1=np.arctan(x+dec)
    y2=np.arctan(-x+dec)
    return y1+y2

y=buildSignal()
plt.plot(y);

Whats we called contour are the fast variations of this signal. Let's us compute its discrete first and second derivation:

In [ ]:
y_decay_right=np.zeros_like(y)
y_decay_right[1:]=y[:-1]

yDiff =  (y - y_decay_right)

plt.title("discrete first derivative")
plt.plot(yDiff);

In [ ]:
# "Théorème des Sauts"
y_decay_left=np.zeros_like(y)
y_decay_left[:-1]=y[1:]

y_diff2=-2*y+y_decay_left+y_decay_right
plt.title("discrete second derivative")
plt.plot(y_diff2);

So we have two ways for detecting the contours:

* Detect the extreme values of the first derivative.
* Detect the zero-crossing ot the second derivative.

***To you:***

* $(2\heartsuit)$ Recompute these discrete derivations with convolutions, using `scipy`.
*  $(1\heartsuit)$ Assume that the signal as a duration of 2 seconds, modify the two previous plot, so that the discrete derivative looks like the true derivative.

***A vous:*** Indiquez si `scipy` effectue la convolution informatique, ou mathématique. Quel est son mode par défaut?  

In [ ]:
signal=np.array([1,2,3])
sg.convolve(signal,[1,-1])#mode full

***A vous:*** Idem avec tensorflow

In [ ]:
import tensorflow as tf
x=tf.constant([1,2,3])
x=tf.reshape(x,[1,3,1])
kernel=tf.constant([1,-1])
kernel=tf.reshape(kernel,[2,1,1])

tf.nn.convolution(x, kernel)

###The norm of gradients

In [ ]:
img_grid = imageio.v2.imread("assets_signal/grille.gif")
#commentez la ligne suivante, et observez le message d'erreur plus bas.
img_grid=img_grid[:,:,0]
img_grid.shape,img_grid.dtype,np.max(img_grid)

In [ ]:
plt.imshow(img_grid, cmap='gray');

In [ ]:
"""we compute the horizontal and vertical gradients with convolutions"""
Gx = sg.convolve(img_grid, [[1,0, -1]], "valid")
Gy = sg.convolve(img_grid, [[1],[0], [-1]], "valid")


fig,(ax0,ax1)=plt.subplots(1,2,figsize=(10,5)) #change figsize, to observe some 'aliasing'

ax0.imshow(Gx, cmap='gray')
ax1.imshow(Gy, cmap='gray');

In [ ]:
Gx.dtype


***To you:***

* $(2\heartsuit)$ Compute and plot the norm of the gradient. Let's denote it by $N$.  

*  $(2\heartsuit)$  Choose a good thresold (=seuil) $S$ and plot the places where $N>S$. You would see the contours appear.

* $(2\heartsuit\flat)$ The purpose of this exercise is to emphasize the link between 1D and 2D signals. Plot some vertical and horizontal slices of the grid image, and of these gradients.  Ex: an hozirontal slices of a matrix $M[i,j]$ is $j\to M[i_o,j]$.





### Smoothing before


***To you:*** Try  $(3\heartsuit)$  various gaussian smoothing before the previous contour detection: you would constast this trade-off:

* Strong smoothing: the detection is robust against the noise, but the contours are thick.

* Weak smoothing: sensibility to the noise, but fine localisation of the contour.


###  Angles of  gradients

In [ ]:
x=np.linspace(-1.5,1.5,100)
xx,yy=np.meshgrid(x,x)
disk=((xx**2+yy**2)<1).astype(float)
plt.imshow(disk,cmap="gray");

***To you:***

* $(2\heartsuit)$ Find the edge of the disk above.
* $(2\heartsuit)$  All allong this edge, compute the angle of the gradient. Help: use `np.arctan2(y,x)`
* $(4\heartsuit\flat)$  Make a graphical representation of this angles. you can use arrows or you can convert the angles into a hue.
* Add a very small noise to the disk (ex: gaussian noise).

        disk+=np.random.normal(scale=1e-6,size=disk.shape)

Recompute the angle and comment.

## Cany filter


Cany filters, and its refinement are the best "unsupervised" methodes to detect edges. Recently, the deep-learning allows more accurate detection, but for training the models, we need a huge number of  images where edges are anotated by hand.

There is an already-made function to perform the cany filter:

        from skimage import feature
        edges = feature.canny(image)

But we will re-implement it in detail: the different ideas of this algorithm could help you in many other situations.  

### Find a local max



In [ ]:
signal=np.array([0,1,2,3,2,0,1,2,1,0])
local_max=(signal[1:-1]>=signal[:-2])&(signal[1:-1]>signal[2:])
local_max*1

***To you:*** why do we use a large and a strict inequality? Try to create example of signals where two strict of two large inequality with change the notion of local-maximum.

### How to work inplace






For performance, we use some optimization of numpy:

* `np.add(a,b)` is the same as `a+b`, but its allow more arguments:
* `where=mask` allow to make the sum where the mask indicates `True`
* `out=` allow to precise the matrix where the result of the sum is put. Its allows to work 'in place' = without create a new array.

Actually the arguments `out` and `where` are present in almost all numpy functions.  

In [ ]:
A=np.zeros(10)
B=np.linspace(-1,1,10)

In [ ]:
np.add(A,np.ones([10]),where=(B>0),out=A)

In [ ]:
A

### Other gradient operator

They are  many possible discretisations for the gradient-operator:

In [ ]:
"""sobel, so-chouette"""
def sobel_operators():
    Sx=[[-1,0,1],
        [-2,0,2],
        [-1,0,1]]

    Sy=[[ 1, 2, 1],
        [ 0, 0, 0],
        [-1,-2, -1]]

    sobel=np.stack([Sx,Sy])

    return sobel

""" Prewit: idem as sobel, changing the 2 into 1 (less good)"""


"""Robert: diagonal gradients"""
def robert_operators():
    Rx=[[1,0],
        [0,-1]]

    Ry=[[ 0,1],
        [-1,0]]

    R=np.stack([Rx,Ry])

    return R

In [ ]:
print(sobel_operators())

There are also the compass operators which compute the gradient in the 8 natural directions (North, North-West, West, ...):

In [ ]:
def getCompassOperators(kind:str):
    res=np.empty([8,3,3],dtype=np.int64)

    if kind=="kirsch":
        conv_mask = np.array([5,  5, 5,-3,-3,-3,-3,-3])
    elif kind=="robinson2":
        conv_mask = np.array([1,2,1,0,-1,-2,-1,0])
    elif kind=="robinson1":
        conv_mask = np.array([1,1,1,0,-1,-1,-1,0])


    for i in range(8):
        oneDir=np.zeros(9,dtype=np.int64)
        oneDir[[0,1,2,5,8,7,6,3]]=conv_mask
        "to turn the coef"
        conv_mask=np.concatenate([conv_mask[1:],conv_mask[0:1]])
        res[i,:,:]=oneDir.reshape([3,3])

    return res


In [ ]:
getCompassOperators("robinson1")

***To you:*** $(1\heartsuit)$ Associate to each operator of the list above its direction (North,North-West,...). Rem: you probably have to make an arbitrary choice.


***To you:*** $(2\heartsuit)$: when you change from an operator to another, the magnitude of the gradients can change a lot. This is a bit enoying, because you have to adapt the threshold. Explain how modify these operator to limitate these amplitude variation.  

### Cany filter


The principle is to find the places where the gradients reach a local maximum when we travel in the image folloging the 4 possible directions:


In [ ]:
def create_simple_img(choice):
    x=np.linspace(0,1,13)
    xx,yy=np.meshgrid(x,x)

    if choice==0:
        img_simple=((xx<0.5)&(yy<0.5))*10
    elif choice==1:
        img_simple=(xx>yy)*10
    else:
        img_simple=(np.abs(xx-1/2)+np.abs(yy-1/2)<0.3)*10

    return img_simple


In [ ]:
img_simple=create_simple_img(2)
print(img_simple)

To well understant the following, do not hesitate to run the programs this the 2 variants of `img_simple`


We compute the gradients accoring to the 4 possible directions:

In [ ]:
operators=getCompassOperators("robinson2")
all_grad = np.array([sg.convolve(img_simple, operators[i,:,:], "valid") for i in range(4)])
all_grad.shape

In [ ]:
all_grad[0,:,:] #gradients ↑

In [ ]:
all_grad[1,:,:] #gradients ↖

In [ ]:
all_grad[2,:,:] #gradients ←

In [ ]:
all_grad[3,:,:] #gradients ↙

For each pixel, we look at the direction which have the greatest gradient.

In [ ]:
""" when the max is reached in several indices, it is the smallest which is chosen. ex:0=np.argmax([10,10,5]) """
direction = np.argmax(np.abs(all_grad), axis=0)
print(direction)

***To you:*** Why so much 0?

We also keep the value of the maximum gradients.

In [ ]:
grad=np.max(np.abs(all_grad),axis=0)
print(grad)

If we just take the previous matrix, we have a detection which is similar to the previous filters (Sobel). But Cany goes further.

It looks for the local maximums of the gradient along the 4 possibles directions.



In [ ]:
Z=np.zeros(direction.shape,dtype=np.int8)

In [ ]:
part=grad[1:-1, :]
np.add(Z[1:-1, :],
       (part>=grad[2:, :]) & (part>grad[:-2, :]),
       where= direction[1:-1, :] == 0,
       out=Z[1:-1, :]
      )
print(Z)

On garde un pixel si

1. C'est un maximal local de `grad` dans la direction ↑
2. Ce maximum local a été atteint en calculant le gradient dans la direction ↑

Cela ne donne que 2 endroits:

    [[ 0  0  0  0  0  0  0  0  0  0  0]
    [ 0  0  0  0 20 20 20  0  0  0  0]
    [ 0  0  0 20 40 40 40 20  0  0  0]
                    ⬆
    [ 0  0 20 40 40 20 40 40 20  0  0]
    [ 0 20 40 40 20  0 20 40 40 20  0]
    [ 0 20 40 20  0  0  0 20 40 20  0]
    [ 0 20 40 40 20  0 20 40 40 20  0]
    [ 0  0 20 40 40 20 40 40 20  0  0]
                    ⬇
    [ 0  0  0 20 40 40 40 20  0  0  0]
    [ 0  0  0  0 20 20 20  0  0  0  0]
    [ 0  0  0  0  0  0  0  0  0  0  0]]

On fait idem avec les 3 autres directions:

In [ ]:
part=grad[1:-1, 1:-1]
np.add(Z[1:-1, 1:-1],
       (part>= grad[:-2, 2:]) & (part>grad[2:, :-2]),
       where=direction[1:-1, 1:-1] == 3,
       out=Z[1:-1, 1:-1])
print(Z)

In [ ]:
part=grad[:, 1:-1]
np.add(Z[:,1:-1],
       (part >= grad[:, 2:]) & (part>grad[:, :-2]),
       where=direction[:, 1:-1] == 2,
       out=Z[:, 1:-1])
print(Z)

In [ ]:
part=grad[1:-1, 1:-1]
np.add(Z[1:-1, 1:-1],
       (part >= grad[2:, 2:]) & (part>grad[:-2, :-2]),
       where=direction[1:-1, 1:-1] == 1,
       out=Z[1:-1, 1:-1]
      )
print(Z)

In [ ]:
plt.imshow(Z,cmap="gray");

***To you:***

*    Why did we use only the 4 first Robinson operators (on 8 possible)?


*    try the orther simple images.

* Try other compass operators

In [ ]:
img_simple=create_simple_img(1)
print(img_simple)

### Hysteresis Thresholding


The edge detection is not finished: if we keep all the pixels such that `Z>0`, we would also detect very light edges: some edges that we would do not detect with our eyes. Indeed: `Z` is computed from local maximums, and not from the absolute value of the gradients.


The Cany filter use the "hysteresis thresholding" which require two thresholds: `low_thr` and `high_thr` with  `0<low_thr<high_thr`.


*  pixels marked by the `Z` matrix,   such that  `grad>=high_thr` are marked as strong edges, and are  keept.
* pixels such that `grad<=low_thr` are rejected
* for pixels marked by the `Z` matrix,   such that  `low_thr<grad<high_thr`, we look at their neighbours: if one of the neighbour is a strong edge, then the are keept, if not, they are rejected.


***To you:***   Perhaps could you use an IDE (ex: pycharm, VS code) for the following exercise.

* $(4\heartsuit)$ Wrap all the previous code in a function. Test it.
* $(8\heartsuit)$ Add the final selection with thresholds. Test different  thresholds. The choice of such thresholds is delicate and depend on the image. In general, the low one is about half the high one.
* $(4\heartsuit)$ Add also a preliminary gaussian smoothing, and test various `sigma`.
* $(2\heartsuit)$ Explain the link between the args of your function and the args of the `scipy` that you get with:

        from skimage import feature
        feature.canny?





### Color images

Here is 3 ways to use cany filter on RGB  images:

* Change the color image into a gray one. But you can miss some edges.
* Make the filtering for each channel, and sum the detected edges. This work well.
* The optimal way is to consider mutli-dimensionnal gradients. A famous one is:
$$
\begin{pmatrix}
R^2_i+G^2_i+B^2_i & R_i R_j+G_iG_j+B_iB_j\\
R_iR_j+G_iG_j+B_iB_j & R^2_j+G^2_j+B^2_j
\end{pmatrix}
$$
where

* $R_i [i,j]= R[i+1,j]-R[i,j]$ (the discrete gradient of the red-channel in the direction →)
* $R_j [i,j]= R[i,j+1]-R[i,j]$
* ...



At each point `i,j` we take

* `grad[i,j]` as the greatest eigen value of the previous matrix
* `direction[i,j]` as  the direction of the eigen vector associated the the greatest eigen value, that we round so its belongs to the 4 possible main directions.

The following of the algorithm is the same: we keep only the local max of `grad[i,j]` in the direction `direction[i,j]` ...


***To you:*** $(10\heartsuit\flat)$ Try this if you have time enough. To extract the greatest eigen value, use the `svd` (see below).



In [ ]:
""" for symetric matrices, the svd decomposition gives  eigen vectors and values, sorted in the decreasing ordre """
mat=np.array([[2,-3],[-3,5]])
V,S,_=np.linalg.svd(mat)
v0=V[:,0]
print(mat@v0)
print(S[0]*v0)

## Second order filter $\flat$

The Laplacian is a generalisation of the second order derivative. Ex: for a function $f$ with two variables, the laplacian $\Delta f$ is:
$$
\Delta f  = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
$$
Here are some possible discretizations of the Laplacian:

In [ ]:
"""Discrete Laplacian 4"""
laplace1 = [[0, 1, 0],
            [1, -4, 1],
            0, 1, 0]]

"""Discrete Laplacian 8"""
laplace2 = [[1, 1, 1],
            [1, -8, 1],
            [1, 1, 1]]

Let's denote by $\ell$ the laplacian of the image, which we can optain by the convolution between the image and one of the previous operator. Then let's write:
$$
L_{i,j} = (\ell_{i,j},\ell_{i+1,j},\ell_{i-1,j},\ell_{i,j+1},\ell_{i,j-1},\ell_{i+1,j+1},\ell_{i+1,j-1},\ell_{i-1,j+1},\ell_{i-1,j-1},)
$$
i.e the nine values of the laplacian around $i,j$.

The pixel $i,j$ belongs to an edge when:
$$
\max(L_{i,j})>0 \text{ and } \min(L_{i,j})<0  \text{ and } \max(L_{i,j})-\min(L_{i,j}) > \text{threshold}.
$$


***To you:*** $(10\heartsuit\flat)$ Try this if you have time enough.